# 1

In [34]:
# this array stores the task names (1, 2, ..., 18)
tasks = []
for i = 1:18
    push!(tasks, i)
end

# this dictionary stores the project durations
dur = [2, 16, 9, 8, 10, 6, 2, 2, 9, 5, 3, 2, 1, 7, 4, 3, 9, 1]
duration = Dict(zip(tasks,dur))

# this dictionary stores the projects that a given project depends on (ancestors)
pre = ( [], [1], [2], [2], [3], [4,5], [4], [6], [4,6], [4], [6], [9], [7], [2], [4,14], [8,11,14], [12], [17])
pred = Dict(zip(tasks,pre));

using JuMP,Clp
m = Model(solver=ClpSolver())

@variable(m, tstart[tasks])

for i in tasks
    for j in pred[i]
        @constraint(m, tstart[i] >= tstart[j] + duration[j])
    end
end

@constraint(m, tstart[1] == 0)
@objective(m, Min, tstart[18] + duration[18]) 

solve(m)
println(getvalue(tstart)) # total duration is start time of last task + duration of last task.
println("minimum duration: ", getobjectivevalue(m))

start time:tstart: 1 dimensions:
[ 1] = -0.0
[ 2] = 2.0
[ 3] = 18.0
[ 4] = 18.0
[ 5] = 27.0
[ 6] = 37.0
[ 7] = 26.0
[ 8] = 43.0
[ 9] = 43.0
[10] = 26.0
[11] = 43.0
[12] = 52.0
[13] = 28.0
[14] = 18.0
[15] = 26.0
[16] = 46.0
[17] = 54.0
[18] = 63.0
minimum duration: 64.0


# 2

In [32]:
# this array stores the task names (1, 2, ..., 18)
tasks = []
for i = 1:18
    push!(tasks, i)
end

# this dictionary stores the project durations
dur = [2, 16, 9, 8, 10, 6, 2, 2, 9, 5, 3, 2, 1, 7, 4, 3, 9, 1]
duration = Dict(zip(tasks,dur))

# this dictionary stores the projects that a given project depends on (ancestors)
pre = ([], [1], [2], [2], [3], [4,5], [4], [6], [4,6], [4], [6], [9], [7], [2], [4,14], [8,11,14], [12], [17])
pred = Dict(zip(tasks,pre));

max_reduction =  [0,  3,  1,  2,  2,  1, 1, 0,  2,  1,  1, 0, 0,  2,  2, 1,  3, 0]  # max reduction (in weeks)
cost_reduction = [0, 30, 26, 12, 17, 15, 8, 0, 42, 21, 18, 0, 0, 22, 12, 6, 16, 0]  # cost of reduction ($1,000/week)
bonus_amount = 30     # bonus for expediting the project ($1,000/week )

using JuMP, Clp
m = Model(solver=ClpSolver())

@variable(m, tstart[tasks])
@variable(m, cost[tasks] >= 0)
@variable(m, reduction_days[tasks] >= 0)

for i in tasks # i is one day after j
    for j in pred[i]
        @constraint(m, reduction_days[j] <= max_reduction[j])
        @constraint(m, tstart[i] >= tstart[j] + duration[j] - reduction_days[j])
        @constraint(m, cost[j] >= reduction_days[j]*cost_reduction[j])
    end
end

@constraint(m, tstart[1] == 0)

@objective(m, Max, bonus_amount*(66-(tstart[18] + duration[18]))-sum(cost))

solve(m)
println(getvalue(tstart))
println("Finish: ",getvalue(tstart[18]+1))
println("Maximum profit: ", getobjectivevalue(m))

tstart: 1 dimensions:
[ 1] = -0.0
[ 2] = 2.0
[ 3] = 15.0
[ 4] = 15.0
[ 5] = 23.0
[ 6] = 31.0
[ 7] = 23.0
[ 8] = 36.0
[ 9] = 36.0
[10] = 23.0
[11] = 36.0
[12] = 45.0
[13] = 25.0
[14] = 15.0
[15] = 23.0
[16] = 39.0
[17] = 47.0
[18] = 53.0
Finish: 54.0
Maximum profit: 147.0
